In [16]:
from pdf2image import convert_from_path, convert_from_bytes

import layoutparser as lp 

import matplotlib.pyplot as plt
%matplotlib inline 

import pandas as pd
import numpy as np
import cv2
import pytesseract
from detectron2 import model_zoo

from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [17]:
images = convert_from_bytes(open('/Volumes/DATA_LAKE/bison_bison/MTAuMjE3NC8xODc0ODM5MjAwOTAzMDEwMDI0.pdf', 'rb').read())

In [22]:
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3550>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F33D0>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3790>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3760>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3370>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3670>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3700>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3460>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F31F0>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F3190>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542F32E0>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x7FC4542

In [3]:
import os
files = os.listdir()

In [4]:
[file for file in files if file.split('.')[1] == 'pdf']

['MTAuMjExMS8xNTUxLTUwMVgoMjAwOCkzMFsyOlRNQVRSR10yLjAuQ087Mg.pdf',
 'MTAuMjE3NC8xODc0ODM5MjAwOTAzMDEwMDI0.pdf']

# OCR

In [23]:
model = lp.Detectron2LayoutModel(
            config_path ='lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config', # In model catalog
            label_map   = {0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )

In [26]:
#loop through each page
for image in images:
    ocr_agent = lp.ocr.TesseractAgent()
    
    stop_words = ['Table', 'Fig', 'Acknowledgements', '©']
    
    image = np.array(image)
    
    layout = model.detect(image)

    text_blocks = lp.Layout([b for b in layout if b.type == 'Text']) #loop through each text box on page.
    
    for block in text_blocks:
        segment_image = (block
                        .pad(left=5, right=5, top=5, bottom=5)
                        .crop_image(image))
        text = ocr_agent.detect(segment_image)
        block.set(text=text, inplace=True)
    
    
    for i, txt in enumerate(text_blocks.get_texts()):
        
        for word in stop_words:
            #filter out text blocks related to copyrights/table descriptions/figure descriptions etc.
            if word in txt[:10]:
                pass
            else:
                my_file = open("MTAuMjE3NC8xODc0ODM5MjAwOTAzMDEwMDI0.txt","a+")
                my_file.write(txt)
    
    
    

KeyboardInterrupt: 

In [32]:
for i, txt in enumerate(text_blocks.get_texts()):

    for word in stop_words:
        #filter out text blocks related to copyrights/table descriptions/figure descriptions etc.
        if word in txt[:10]:
            pass
        else:
            print(txt)
            
    #print(txt)

EINP exists in a transitional zone between the prairie and
boreal forest with vegetation classified as dry mixedwood
boreal forest [21]. Trembling aspen (Populus tremuloides) is
the dominant tree species and commonly co-exists with bal-
sam poplar (Populus trichocarpa) or white birch (Betula
papyrifera), with additional small areas of white spruce
(Picea glauca) and black spruce (Picea mariana) in the
northern part of the park. Shrub understories consist of
beaked hazel (Corylus cornuta), dogwood (Cornus stolonif-
era), chokecherry (Prunus virginiana), willow (Salix spp.),
prickly rose (Rosa acicularis), raspberry (Rubus spp.) and
other shrubs. Common native grasses are Agropyron spp.,
Elymus innovatus and Oryzopsis asperifolia, with introduced
species being Trifolium spp., Bromus spp. and Poa spp.,
whereas wet meadows are dominated by Calamagrostis ca-
nadensis and hydric Carex spp. often with an overstory of
Salix spp. [58]. Vegetation composition is further described
by Bork et al. 

In [14]:
def parse():
    files = os.listdir()
    # grab only pdf docs.
    print("FILES:", files)
    filtered_files = [file for file in files if file.split('.')[
        1] == 'pdf']

    model = lp.Detectron2LayoutModel(
        config_path='lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',  # In model catalog
        label_map={0: "Text", 1: "Title", 2: "List",
                   3: "Table", 4: "Figure"},  # In model`label_map`
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8]  # Optional
    )

    ocr_agent = lp.ocr.TesseractAgent()

    for file in filtered_files:
        filename = file.split('.')[0]

        # We have to convert each page of the PDF into images.
        images = convert_from_bytes(
            open(file, 'rb').read())

        for image in images:

            print("STEP1")

            stop_words = ['Table', 'Fig', 'Acknowledgements', '©']

            image = np.array(image)

            # detect the layout on the page(image)
            layout = model.detect(image)

            print("STEP2")

            # loop through each text box on page(image).
            text_blocks = lp.Layout(
                [b for b in layout if b.type == 'Text'])

            for block in text_blocks:
                print('analyzed block1')
                segment_image = (block
                                 .pad(left=5, right=5, top=5, bottom=5)
                                 .crop_image(image))
                text = ocr_agent.detect(segment_image)
                block.set(text=text, inplace=True)
                print('analyzed block2')

                for i, txt in enumerate(block.get_texts()):

                    for word in stop_words:
                        # filter out text blocks related to copyrights/table descriptions/figure descriptions etc.
                        if word in txt[:10]:
                            pass
                        else:
                            my_file = open(filename + ".txt", "a+")
                            my_file.write(txt)
                            print('FILE SAVED')

In [15]:
parse()

FILES: ['.DS_Store', 'MTAuMjExMS8xNTUxLTUwMVgoMjAwOCkzMFsyOlRNQVRSR10yLjAuQ087Mg.pdf', 'Cascadetab.ipynb', 'HorizonInterview.ipynb', 'img_tbl_items.png', 'MTAuMjExMS8xNTUxLTUwMVgoMjAwOCkzMFsyOlRNQVRSR10yLjAuQ087Mg.txt', 'layout-parser.ipynb', 'PDF2img.ipynb', '.ipynb_checkpoints', 'MTAuMjE3NC8xODc0ODM5MjAwOTAzMDEwMDI0.pdf', '.git', 'Advanced_Analytics_Assessment_2021_04_01.ipynb']
STEP1
STEP2
analyzed block1
analyzed block2


AttributeError: 'TextBlock' object has no attribute 'get_texts'